In [2]:
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import h5py
from subprocess import check_output

from my_classesnfunc import *

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, merge, Input, Lambda, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras import backend as K

C:\Users\Kuba\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    SIZE = input_dim[0]
    seq = Sequential()
    seq.add(Conv2D(filters=15,
                       kernel_size=(3, 3),
                       activation='relu',
                       input_shape = input_dim,
                       padding = 'same'))
    seq.add(MaxPooling2D())

    seq.add(Conv2D(filters=30,
                       kernel_size=(5, 5),
                       activation='relu',
                       padding = 'same'))
    seq.add(MaxPooling2D())
    seq.add(Flatten())
        
    seq.add(Dropout(0.1))
    seq.add(Dense(int((SIZE/4)**2), activation='relu'))
    return seq

In [3]:
hf = h5py.File('tr_gr_64.h5', 'r')
x = hf.get('x')
y = hf.get('y')


In [4]:
x = np.array(x)
y = np.array(y)
print(np.shape(x))
print(np.shape(y))
hf.close()

(9040, 64, 64)
(9040,)


In [5]:
lohe = LabelOneHotEncoder()
y_cat = lohe.fit_transform(y)

In [5]:
shape = np.shape(x)
x = x.reshape((shape[0],shape[1],shape[2],1))
input_dim = (shape[-3],shape[-2],shape[-1])
SIZE = shape[2]
n_el = shape[0]

In [8]:
x = x.astype('float32')
x /= 255


In [117]:
base_network = create_base_network(input_dim)

input_a = Input(shape=input_dim)
input_b = Input(shape=input_dim)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
prediction = Dense(units=1,
                       activation='sigmoid',
                       )(distance)

model = Model(inputs=[input_a, input_b], outputs=prediction)

In [118]:
optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [ ]:
x_flat = x.reshape((n_el,SIZE**2))
dists = np.zeros((n_el,n_el))
for i in range(n_el):
    for j in range(n_el):
        if i >= j:
            dists[i,j] = 0
            continue
        vec1 = x_flat[i]
        vec2 = x_flat[j]
        dists[i,j] = compute_distances_no_loops(vec1, vec2)